In [21]:
# Получение данных
import pandas as pd

task = ['max', 'min', 'min'] # Задача оптимизации
file_name = './Hysys/data/hysys729_v2.csv'  # Откуда брать файл
data_source = pd.read_csv(file_name, sep=',', index_col=0,
                          skiprows=[1]  # Пропуск единиц измерения
                          ).dropna(
    axis=1, how='all')  # Чтение файла


# pd.options.display.float_format = '{:,.2f}'.format # Если нужно ограничить вывод до 2 знаков после запятой
column_names = data_source.columns.values  # Получение названий столбцов

# Разбиение таблицы на параметры и критерии
params = data_source.iloc[:, 0:6].astype(float)
criteria1_column = data_source.iloc[:, 6].astype(float) 
criteria2_column = data_source.iloc[:, 7].astype(float) 
criteria3_column = data_source.iloc[:, 8].astype(float)

print('Размер таблицы:', data_source.shape)

ERROR_VALUE = -32767
if data_source[data_source == ERROR_VALUE].count().sum() > 0 or data_source.isna().sum().sum()>0:
    print('В таблице есть битые данные')
    print(data_source[data_source == ERROR_VALUE].count())
    print(data_source.isna().sum())


Размер таблицы: (729, 9)


In [22]:
# Линейная интерполяция
from scipy.interpolate import LinearNDInterpolator
import numpy as np

# Создание точек для интерполяции
points = np.array(params.values)

PRINT_PROGRESS = False  # Вывод прогресса
SAVE_RESULTS = False  # Сохранение результата

# Псевдовеса,если нужно максимизировать критерий, то умножаем его на -1
weight = [-1 if s == 'max' else 1 for s in task]

# Границы параметров
lower_bound = params.min().to_numpy()
upper_bound = params.max().to_numpy()

# Создание интерполяторов
print('Расчет интерполяторов')
print('Расчет интерполятора:', column_names[6])
criteria1 = LinearNDInterpolator(points, criteria1_column.values * weight[0])
print('Расчет интерполятора:', column_names[7])
criteria2 = LinearNDInterpolator(
    points, criteria2_column.values * weight[1])
print('Расчет интерполятора:', column_names[8])
criteria3 = LinearNDInterpolator(
    points, criteria3_column.values * weight[2])
print('Расчет интерполяторов завершен')

test_point = params.sample(1).values[0]
print('Проверка работы интерполяторов в точке', test_point) # Главное, чтобы просто nan не выдавало
print(column_names[6], test_point, criteria1(test_point))
print(column_names[7], test_point, criteria2(test_point))
print(column_names[8], test_point, criteria3(test_point))

bounds = pd.DataFrame([lower_bound, upper_bound],
                      columns=column_names[0:6],
                      index=['Нижняя граница', 'Верхняя граница'])
bounds.transpose()


Расчет интерполяторов
Расчет интерполятора: 10 - Master Comp Molar Flow (diM-Ether)
Расчет интерполятора: 14 - Master Comp Molar Flow (Methanol)
Расчет интерполятора: SPRDSHT-1 - B1:
Расчет интерполяторов завершен
Проверка работы интерполяторов в точке [ 170.  250.  360.  110.   60. 2000.]
10 - Master Comp Molar Flow (diM-Ether) [ 170.  250.  360.  110.   60. 2000.] [-128.464491]
14 - Master Comp Molar Flow (Methanol) [ 170.  250.  360.  110.   60. 2000.] [2.198327]
SPRDSHT-1 - B1: [ 170.  250.  360.  110.   60. 2000.] [48309525.309864]


,Нижняя граница,Верхняя граница
4 - Temperature,140.0,170.0
5 - Temperature,240.0,260.0
6 - Temperature,340.0,360.0
8 - Temperature,90.0,110.0
VLV-100 - Pressure Drop,20.0,60.0
P-100 - Delta P,1000.0,2000.0


In [23]:
# Однокритериальная оптимизация

import pandas as pd
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
# Алгоритмы
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
from pymoo.algorithms.soo.nonconvex.de import DE # Алгоритм дифференциальной эволюции, если Хука-Дживса выдает nan


class Find_best_c1 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria1(x)


class Find_best_c2 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria2(x)


class Find_best_c3 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria3(x)

soo_c1 = minimize(Find_best_c1(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
soo_c2 = minimize(Find_best_c2(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
soo_c3 = minimize(Find_best_c3(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)

# Вывод в виде таблицы
single_criteria_optimization_df = pd.DataFrame(
    columns=column_names[0:6],
    data=[soo_c1.X, soo_c2.X, soo_c3.X],
    index=column_names[6:9]
)
single_criteria_optimization_df['Значение критерия'] = [
    weight[0] * soo_c1.F[0], weight[1] * soo_c2.F[0], weight[2] * soo_c3.F[0]]

print('Результаты однокритериальной оптимизации:')
single_criteria_optimization_df.transpose()


Результаты однокритериальной оптимизации:


,10 - Master Comp Molar Flow (diM-Ether),14 - Master Comp Molar Flow (Methanol),SPRDSHT-1 - B1:
4 - Temperature,140.00000,140.000000,1.400000e+02
5 - Temperature,260.00000,260.000000,2.600000e+02
6 - Temperature,360.00000,360.000000,3.500000e+02
8 - Temperature,110.00000,110.000000,1.100000e+02
VLV-100 - Pressure Drop,20.00000,20.000000,2.000000e+01
P-100 - Delta P,2000.00000,1000.000000,1.500000e+03
Значение критерия,128.56923,1.989665,3.014509e+07


In [26]:
# Оптимизация на основе маргинальных решений

from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
# Алгоритм дифференциальной эволюции
from pymoo.algorithms.soo.nonconvex.de import DE
import numpy as np


def marg_objective_function(x):
    '''Маргинальная целевая функция'''
    diff_c1 = soo_c1.F - criteria1(x)
    diff_c2 = soo_c2.F - criteria2(x)
    diff_c3 = soo_c3.F - criteria3(x)

    return np.sqrt(diff_c1**2 + diff_c2**2 + diff_c3**2)


# Постановка задачи однокритериальной оптимизации
class Marg_optimization(Problem):
    def __init__(self) -> None:
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = marg_objective_function(x)


# Запуск оптимизации
try:
    res_PS = minimize(
        Marg_optimization(), algorithm=PatternSearch(), seed=1)

    # Вывод результатов в виде таблицы
    margin_optimization_df = pd.DataFrame(
        data=(
            res_PS.X.tolist() + (weight[0] * criteria1(res_PS.X)).tolist() + (weight[1] * criteria2(res_PS.X)).tolist() + (weight[2] * criteria3(res_PS.X)).tolist()),
        index=column_names[0:9],
    )
except Exception as e:
    print('Ошибка маргинальной оптимизации')
    print(e)

print('Результаты маргинальной оптимизации')
margin_optimization_df


Результаты маргинальной оптимизации


,0
4 - Temperature,1.400000e+02
5 - Temperature,2.600000e+02
6 - Temperature,3.500000e+02
8 - Temperature,1.100000e+02
VLV-100 - Pressure Drop,2.000000e+01
P-100 - Delta P,1.500000e+03
10 - Master Comp Molar Flow (diM-Ether),1.282883e+02
14 - Master Comp Molar Flow (Methanol),2.622294e+00
SPRDSHT-1 - B1:,3.014509e+07


In [30]:
# Многокритериальная оптимизация
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Алгоритмы оптимизации
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.age import AGEMOEA


# Постановка задачи
class Multi_objective_optimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0,
                         # Минимальные значения параметров
                         xl=lower_bound,
                         # Максимальные значения параметров
                         xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = criteria1(x)
        f2 = criteria2(x)
        f3 = criteria3(x)
        out["F"] = np.column_stack([f1, f2, f3])


problem = Multi_objective_optimization()

try:
    print('Многокритериальная оптимизация (Метод NSGA2)')
    res_NSGA2 = minimize(problem, algorithm=NSGA2(), seed=1,
                         verbose=PRINT_PROGRESS)  # Вывод прогресса)

    # Сохранение результатов
    res_NSGA2_df = pd.DataFrame(res_NSGA2.X)
    res_NSGA2_df['c1'] = weight[0] * res_NSGA2.F[:, 0]
    res_NSGA2_df['c2'] = weight[1] * res_NSGA2.F[:, 1]
    res_NSGA2_df['c3'] = weight[2] * res_NSGA2.F[:, 2]
    res_NSGA2_df.columns = column_names

    if SAVE_RESULTS:
        res_NSGA2_df.to_csv('NSGA2_Results.csv', sep=',', index=False)

except Exception as e:
    print('Ошибка NSGA2')
    print(e)

try:
    print('Многокритериальная оптимизация (Метод AGEMOEA)')
    res_AGEMOEA = minimize(problem, algorithm=AGEMOEA(),
                           seed=1, verbose=PRINT_PROGRESS,
                           termination=('time', '00:01:00') # Не позволяем наглеть
                           )

    # Сохранение результатов
    res_AGEMOEA_df = pd.DataFrame(res_AGEMOEA.X)
    res_AGEMOEA_df['c1'] = weight[0] * res_AGEMOEA.F[:, 0]
    res_AGEMOEA_df['c2'] = weight[1] * res_AGEMOEA.F[:, 1]
    res_AGEMOEA_df['c3'] = weight[2] * res_AGEMOEA.F[:, 2]
    res_AGEMOEA_df.columns = column_names

    if SAVE_RESULTS:
        res_AGEMOEA_df.to_csv('AGEMOEA_Results.csv', sep=',', index=False)
        
except Exception as e:
    print('Ошибка AGEMOEA')
    print(e)


Многокритериальная оптимизация (Метод NSGA2)
Многокритериальная оптимизация (Метод AGEMOEA)


In [31]:
res_NSGA2_df

,4 - Temperature,5 - Temperature,6 - Temperature,8 - Temperature,VLV-100 - Pressure Drop,P-100 - Delta P,10 - Master Comp Molar Flow (diM-Ether),14 - Master Comp Molar Flow (Methanol),SPRDSHT-1 - B1:
0,140.0,260.0,360.000000,110.0,20.000000,1000.000000,128.569848,1.989665,3.035407e+07
1,140.0,260.0,349.999200,110.0,20.000000,1540.707781,128.288254,2.622380,3.014524e+07
2,140.0,260.0,349.999200,110.0,20.000000,1499.322818,128.288237,2.622361,3.014516e+07
3,140.0,260.0,355.007988,110.0,20.000000,1692.542369,128.428927,2.306064,3.024769e+07
4,140.0,260.0,360.000000,110.0,21.891417,1011.552483,128.569776,1.989784,3.035367e+07


In [32]:
res_AGEMOEA_df

,4 - Temperature,5 - Temperature,6 - Temperature,8 - Temperature,VLV-100 - Pressure Drop,P-100 - Delta P,10 - Master Comp Molar Flow (diM-Ether),14 - Master Comp Molar Flow (Methanol),SPRDSHT-1 - B1:
0,140.0,260.0,353.556876,110.000000,24.813262,1510.619897,128.388151,2.397530,3.021802e+07
1,140.0,260.0,359.465024,109.999999,25.239343,1478.976529,128.554021,2.024449,3.033866e+07
2,140.0,260.0,351.853611,110.000000,20.055709,1500.106121,128.340303,2.505215,3.018291e+07
3,140.0,260.0,356.851290,110.000000,20.000000,1500.000702,128.480574,2.189551,3.028485e+07
4,140.0,260.0,358.536742,109.999999,22.711343,1499.908545,128.527902,2.083033,3.031943e+07
...,...,...,...,...,...,...,...,...,...
95,140.0,260.0,353.212919,109.999999,20.000936,1499.928711,128.378454,2.419359,3.021063e+07
96,140.0,260.0,358.433732,109.999997,20.176165,1425.432959,128.525124,2.089447,3.031789e+07
97,140.0,260.0,357.902969,110.000000,20.123485,1498.766959,128.510095,2.123120,3.030633e+07
98,140.0,260.0,352.313371,110.000000,20.022447,1499.967336,128.353207,2.476176,3.019229e+07
